In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import coo_matrix
from scipy.sparse import csr_matrix
from scipy.sparse import save_npz
from scipy.sparse import load_npz
from datasketch import MinHash
from itertools import combinations
import numpy as np
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import math
import hashlib
from collections import defaultdict
import signal
from joblib import Parallel, delayed


from datasketch import MinHashLSH

In [2]:
# create a user_movie matrix

data = np.load('../../temp_data/user_movie_rating.npy')
data_array = data.astype(int)

# Extract user, movie, and rating data from the loaded records
user_ids, movie_ids, ratings = data[:, 0], data[:, 1], data[:, 2]

# Create a CSR 
user_movie_matrix = csr_matrix((ratings, (user_ids, movie_ids)))


# Load the user-movie ratings data from the npz file
num_users = user_movie_matrix.shape[0]
num_movies = user_movie_matrix.shape[1]

# Minhashing FOR JS

In [3]:
# Step 1: Generate MinHash signatures for each user

# user_movie_matrix = csr_matrix((ratings, (user_ids, movie_ids)))

# Number of hash functions for MinHashing
n = 80

# Create an array to store MinHash signatures for each user
minhash_signatures = []

# Function to generate MinHash signatures for a set of movie ratings
def generate_minhash_signature(ratings):
    minhash = MinHash(num_perm=n)
    for movie_id in ratings.nonzero()[1]:
        minhash.update(str(movie_id).encode('utf-8'))
    return minhash

# Generate MinHash signatures for each user
for user_id in range(num_users):
    user_ratings = user_movie_matrix.getrow(user_id)
    minhash_signature = generate_minhash_signature(user_ratings)
    minhash_signatures.append(minhash_signature)




In [9]:
# Step 2: Generate candidate pairs of users

# Function to hash a part of a MinHash signature
def hash_band(band):
    return hash(tuple(band))

# Create a list to store the candidate pairs
candidate_pairs = []

# For each band
for band in range(b):
    # Create a dictionary to store the buckets
    buckets = defaultdict(list)
    
    # For each user
    for user_id, minhash_signature in enumerate(minhash_signatures):
        # Hash the part of the MinHash signature in this band
        band_signature = minhash_signature.hashvalues[band*r : (band+1)*r]
        bucket_id = hash_band(band_signature)
        
        # Add the user to the corresponding bucket
        buckets[bucket_id].append(user_id)

    # For each bucket
    for bucket in buckets.values():
        # If the bucket contains more than one user
        if len(bucket) > 1:
            # Add all pairs of users in this bucket to the candidate pairs
            for i in range(len(bucket)):
                for j in range(i+1, len(bucket)):
                    candidate_pairs.append((bucket[i], bucket[j]))

# Now, candidate_pairs contains the pairs of users that are potentially similar

In [3]:
# Step 3: Calculate the Jaccard Similarity for each candidate pair
# Function to calculate Jaccard Similarity
def jaccard_similarity(user1, user2):
    # Get the sets of movies that each user has rated
    movies_user1 = set(user_movie_matrix.getrow(user1).nonzero()[1])
    movies_user2 = set(user_movie_matrix.getrow(user2).nonzero()[1])
    
    # Calculate the size of the intersection and the union of the two sets
    intersection = len(movies_user1 & movies_user2)
    union = len(movies_user1 | movies_user2)
    
    # Calculate and return the Jaccard Similarity
    return intersection / union

# # Create a list to store the pairs of users that are similar according to the Jaccard Similarity
# similar_pairs = []

# jaccard_threhshold = 0.5
# # For each candidate pair
# for user1, user2 in candidate_pairs:
#     # Calculate the exact Jaccard Similarity
#     similarity = jaccard_similarity(user1, user2)
    
#     # If the similarity exceeds the threshold
#     if similarity > jaccard_threhshold:
#         # Add the pair to the list of similar pairs
#         similar_pairs.append((user1, user2, similarity))

# # For each pair of similar users
# for user1, user2, similarity in similar_pairs:
#     # Print the pair of users and their similarity
#     print(f'User1: {user1}, User2: {user2}, Similarity: {similarity}')

# # Print the total number of similar pairs
# print(f'Total number of similar pairs: {len(similar_pairs)}')


In [4]:
# Step 4: Calculate the candidate pairs for this combination of h and b
def calculate_pairs(h, b):
    # Calculate the number of rows in each band
    r = h // b

    # Create a list to store the candidate pairs
    candidate_pairs = []

    # For each band
    for band in range(b):
        # Create a dictionary to store the buckets
        buckets = defaultdict(list)
        
        # For each user
        for user_id, minhash_signature in enumerate(minhash_signatures):
            # Hash the part of the MinHash signature in this band
            band_signature = minhash_signature.hashvalues[band*r : (band+1)*r]
            bucket_id = hash_band(band_signature)
            
            # Add the user to the corresponding bucket
            buckets[bucket_id].append(user_id)

        # For each bucket
        for bucket in buckets.values():
            # If the bucket contains more than one user
            if len(bucket) > 1:
                # Add all pairs of users in this bucket to the candidate pairs
                for i in range(len(bucket)):
                    for j in range(i+1, len(bucket)):
                        candidate_pairs.append((bucket[i], bucket[j]))

    return candidate_pairs

In [10]:
# Step 5: Test the algorithm for different values of h and b
# Function to handle the alarm signal
def handler(signum, frame):
    raise Exception("Time is up")

# Set the signal handler
signal.signal(signal.SIGALRM, handler)

# Set the alarm for 3 minutes
signal.alarm(3 * 60)

h_values = [80]  # Only test with h=80
b_values = [5, 8, 10, 11, 13, 15, 20]   # Example values for b
jaccard_threshold = 0.5
# Initialize an empty list to store the results
results = []


# Iterate over all combinations of h and b
for h in h_values:
    for b in b_values:
        # Reset the alarm for 3 minutes
        signal.alarm(3 * 60)
        
        # Initialize an empty list to store the results for this combination
        results_for_combination = []
        try:
            # Calculate the candidate pairs for this combination of h and b
            pairs = calculate_pairs(h, b)
            # For each candidate pair
            for user1, user2 in pairs:
                # Calculate the exact Jaccard Similarity
                similarity = jaccard_similarity(user1, user2)
                # If the similarity exceeds the threshold
                if similarity > jaccard_threshold:
                    # Add the pair to the list of similar pairs
                    results_for_combination.append((user1, user2, similarity))
        except Exception as e:
            print(f"Stopped calculation for h={h} and b={b} due to time limit")
        finally:
            # Print the number of similar pairs for this combination
            print(f"For h={h} and b={b}, number of similar pairs: {len(results_for_combination)}")
            # Add the results for this combination to the overall results
            results.extend(results_for_combination)

# Print the total number of similar pairs
print(f'Total number of similar pairs: {len(results)}')

For h=80 and b=5, number of similar pairs: 0
For h=80 and b=8, number of similar pairs: 22
For h=80 and b=10, number of similar pairs: 77
For h=80 and b=11, number of similar pairs: 158
Stopped calculation for h=80 and b=13 due to time limit
For h=80 and b=13, number of similar pairs: 11
Stopped calculation for h=80 and b=15 due to time limit
For h=80 and b=15, number of similar pairs: 21
Stopped calculation for h=80 and b=20 due to time limit
For h=80 and b=20, number of similar pairs: 0
Total number of similar pairs: 289


Now for different number of permutations
1. Call the minhash again for new n value
3. Step 5. with new values

## N = 100

In [5]:
# Step 1: Generate MinHash signatures for each user

# user_movie_matrix = csr_matrix((ratings, (user_ids, movie_ids)))

# Number of hash functions for MinHashing
n = 100

# Create an array to store MinHash signatures for each user
minhash_signatures = []

# Function to generate MinHash signatures for a set of movie ratings
def generate_minhash_signature(ratings):
    minhash = MinHash(num_perm=n)
    for movie_id in ratings.nonzero()[1]:
        minhash.update(str(movie_id).encode('utf-8'))
    return minhash

# Generate MinHash signatures for each user
for user_id in range(num_users):
    user_ratings = user_movie_matrix.getrow(user_id)
    minhash_signature = generate_minhash_signature(user_ratings)
    minhash_signatures.append(minhash_signature)




In [7]:
# Step 5: Test the algorithm for different values of h and b N = 100
# Function to handle the alarm signal
def handler(signum, frame):
    raise Exception("Time is up")

# Set the signal handler
signal.signal(signal.SIGALRM, handler)

# Set the alarm for 3 minutes
signal.alarm(3 * 60)

h_values = [100]  # Only test with h=80
b_values = [5, 10, 11, 12, 14, 16]   # Example values for b
jaccard_threshold = 0.5
n = 100
# Initialize an empty list to store the results
results = []


# Iterate over all combinations of h and b
for h in h_values:
    for b in b_values:
        # Calculate r based on the new value of b
        r = n // b

        # Reset the alarm for 3 minutes
        signal.alarm(3 * 60)

        # Initialize an empty list to store the candidate pairs
        candidate_pairs = []
        for band in range(b):
            # Create a dictionary to store the buckets
            buckets = defaultdict(list)
            
            # For each user
            for user_id, minhash_signature in enumerate(minhash_signatures):
                # Hash the part of the MinHash signature in this band
                band_signature = minhash_signature.hashvalues[band*r : (band+1)*r]
                bucket_id = hash_band(band_signature)
                
                # Add the user to the corresponding bucket
                buckets[bucket_id].append(user_id)

            # For each bucket
            for bucket in buckets.values():
            # If the bucket contains more than one user
                if len(bucket) > 1:
                    # Add all pairs of users in this bucket to the candidate pairs
                    for i in range(len(bucket)):
                        for j in range(i+1, len(bucket)):
                            candidate_pairs.append((bucket[i], bucket[j]))
        # Initialize an empty list to store the results for this combination
        results_for_combination = []
        try:
            # For each candidate pair
            for user1, user2 in candidate_pairs:
                # Calculate the exact Jaccard Similarity
                similarity = jaccard_similarity(user1, user2)
                # If the similarity exceeds the threshold
                if similarity > jaccard_threshold:
                    # Add the pair to the list of similar pairs
                    results_for_combination.append((user1, user2, similarity))
        except Exception as e:
            print(f"Stopped calculation for h={h} and b={b} due to time limit")
        finally:
            # Print the number of similar pairs for this combination
            print(f"For h={h} and b={b}, number of similar pairs: {len(results_for_combination)}")
            # Add the results for this combination to the overall results
            results.extend(results_for_combination)

# Print the total number of similar pairs
print(f'Total number of similar pairs: {len(results)}')

For h=100 and b=5, number of similar pairs: 0
For h=100 and b=10, number of similar pairs: 22
For h=100 and b=11, number of similar pairs: 33
For h=100 and b=12, number of similar pairs: 85
Stopped calculation for h=100 and b=14 due to time limit
For h=100 and b=14, number of similar pairs: 160
Stopped calculation for h=100 and b=16 due to time limit
For h=100 and b=16, number of similar pairs: 11
Total number of similar pairs: 311


## N = 120

In [8]:
# Step 1: Generate MinHash signatures for each user

# user_movie_matrix = csr_matrix((ratings, (user_ids, movie_ids)))

# Number of hash functions for MinHashing
n = 120

# Create an array to store MinHash signatures for each user
minhash_signatures = []

# Function to generate MinHash signatures for a set of movie ratings
def generate_minhash_signature(ratings):
    minhash = MinHash(num_perm=n)
    for movie_id in ratings.nonzero()[1]:
        minhash.update(str(movie_id).encode('utf-8'))
    return minhash

# Generate MinHash signatures for each user
for user_id in range(num_users):
    user_ratings = user_movie_matrix.getrow(user_id)
    minhash_signature = generate_minhash_signature(user_ratings)
    minhash_signatures.append(minhash_signature)




In [9]:
# Step 5: Test the algorithm for different values of h and b N = 100
# Function to handle the alarm signal
def handler(signum, frame):
    raise Exception("Time is up")

# Set the signal handler
signal.signal(signal.SIGALRM, handler)

# Set the alarm for 3 minutes
signal.alarm(3 * 60)

h_values = [120]  # Only test with h=80
b_values = [5, 10, 12, 15, 17, 20]   # Example values for b
jaccard_threshold = 0.5
n = 120
# Initialize an empty list to store the results
results = []


# Iterate over all combinations of h and b
for h in h_values:
    for b in b_values:
        # Calculate r based on the new value of b
        r = n // b

        # Reset the alarm for 3 minutes
        signal.alarm(3 * 60)

        # Initialize an empty list to store the candidate pairs
        candidate_pairs = []
        for band in range(b):
            # Create a dictionary to store the buckets
            buckets = defaultdict(list)
            
            # For each user
            for user_id, minhash_signature in enumerate(minhash_signatures):
                # Hash the part of the MinHash signature in this band
                band_signature = minhash_signature.hashvalues[band*r : (band+1)*r]
                bucket_id = hash_band(band_signature)
                
                # Add the user to the corresponding bucket
                buckets[bucket_id].append(user_id)

            # For each bucket
            for bucket in buckets.values():
            # If the bucket contains more than one user
                if len(bucket) > 1:
                    # Add all pairs of users in this bucket to the candidate pairs
                    for i in range(len(bucket)):
                        for j in range(i+1, len(bucket)):
                            candidate_pairs.append((bucket[i], bucket[j]))
        # Initialize an empty list to store the results for this combination
        results_for_combination = []
        try:
            # For each candidate pair
            for user1, user2 in candidate_pairs:
                # Calculate the exact Jaccard Similarity
                similarity = jaccard_similarity(user1, user2)
                # If the similarity exceeds the threshold
                if similarity > jaccard_threshold:
                    # Add the pair to the list of similar pairs
                    results_for_combination.append((user1, user2, similarity))
        except Exception as e:
            print(f"Stopped calculation for h={h} and b={b} due to time limit")
        finally:
            # Print the number of similar pairs for this combination
            print(f"For h={h} and b={b}, number of similar pairs: {len(results_for_combination)}")
            # Add the results for this combination to the overall results
            results.extend(results_for_combination)

# Print the total number of similar pairs
print(f'Total number of similar pairs: {len(results)}')

For h=120 and b=5, number of similar pairs: 0
For h=120 and b=10, number of similar pairs: 5
For h=120 and b=12, number of similar pairs: 27
For h=120 and b=15, number of similar pairs: 108
Stopped calculation for h=120 and b=17 due to time limit
For h=120 and b=17, number of similar pairs: 155
Stopped calculation for h=120 and b=20 due to time limit
For h=120 and b=20, number of similar pairs: 11
Total number of similar pairs: 306


## N = 150

In [10]:
# Step 1: Generate MinHash signatures for each user

# user_movie_matrix = csr_matrix((ratings, (user_ids, movie_ids)))

# Number of hash functions for MinHashing
n = 150

# Create an array to store MinHash signatures for each user
minhash_signatures = []

# Function to generate MinHash signatures for a set of movie ratings
def generate_minhash_signature(ratings):
    minhash = MinHash(num_perm=n)
    for movie_id in ratings.nonzero()[1]:
        minhash.update(str(movie_id).encode('utf-8'))
    return minhash

# Generate MinHash signatures for each user
for user_id in range(num_users):
    user_ratings = user_movie_matrix.getrow(user_id)
    minhash_signature = generate_minhash_signature(user_ratings)
    minhash_signatures.append(minhash_signature)




In [11]:
# Step 5: Test the algorithm for different values of h and b N = 100
# Function to handle the alarm signal
def handler(signum, frame):
    raise Exception("Time is up")

# Set the signal handler
signal.signal(signal.SIGALRM, handler)

# Set the alarm for 3 minutes
signal.alarm(3 * 60)

h_values = [150]  # Only test with h=80
b_values = [5, 10, 15, 21, 25, 30]   # Example values for b
jaccard_threshold = 0.5
n = 150
# Initialize an empty list to store the results
results = []


# Iterate over all combinations of h and b
for h in h_values:
    for b in b_values:
        # Calculate r based on the new value of b
        r = n // b

        # Reset the alarm for 3 minutes
        signal.alarm(3 * 60)

        # Initialize an empty list to store the candidate pairs
        candidate_pairs = []
        for band in range(b):
            # Create a dictionary to store the buckets
            buckets = defaultdict(list)
            
            # For each user
            for user_id, minhash_signature in enumerate(minhash_signatures):
                # Hash the part of the MinHash signature in this band
                band_signature = minhash_signature.hashvalues[band*r : (band+1)*r]
                bucket_id = hash_band(band_signature)
                
                # Add the user to the corresponding bucket
                buckets[bucket_id].append(user_id)

            # For each bucket
            for bucket in buckets.values():
            # If the bucket contains more than one user
                if len(bucket) > 1:
                    # Add all pairs of users in this bucket to the candidate pairs
                    for i in range(len(bucket)):
                        for j in range(i+1, len(bucket)):
                            candidate_pairs.append((bucket[i], bucket[j]))
        # Initialize an empty list to store the results for this combination
        results_for_combination = []
        try:
            # For each candidate pair
            for user1, user2 in candidate_pairs:
                # Calculate the exact Jaccard Similarity
                similarity = jaccard_similarity(user1, user2)
                # If the similarity exceeds the threshold
                if similarity > jaccard_threshold:
                    # Add the pair to the list of similar pairs
                    results_for_combination.append((user1, user2, similarity))
        except Exception as e:
            print(f"Stopped calculation for h={h} and b={b} due to time limit")
        finally:
            # Print the number of similar pairs for this combination
            print(f"For h={h} and b={b}, number of similar pairs: {len(results_for_combination)}")
            # Add the results for this combination to the overall results
            results.extend(results_for_combination)

# Print the total number of similar pairs
print(f'Total number of similar pairs: {len(results)}')

For h=150 and b=5, number of similar pairs: 0
For h=150 and b=10, number of similar pairs: 2
For h=150 and b=15, number of similar pairs: 28
Stopped calculation for h=150 and b=21 due to time limit
For h=150 and b=21, number of similar pairs: 157
Stopped calculation for h=150 and b=25 due to time limit
For h=150 and b=25, number of similar pairs: 11
Stopped calculation for h=150 and b=30 due to time limit
For h=150 and b=30, number of similar pairs: 20
Total number of similar pairs: 218
